In [1]:
import cv2
import open3d as o3d
import numpy as np
import colorsys
import pyransac3d as pyrsc
import copy

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
def visualizer(cloud):

    if type(cloud) == o3d.cpu.pybind.geometry.TriangleMesh:
        vertices = np.asarray(cloud.vertices)
        center_of_mass = np.mean(vertices, axis=0)
    else:
        # point cloud
        center_of_mass = np.mean(np.asarray(cloud.points), axis=0)
    
    # print("com", center_of_mass)
    o3d.visualization.draw_geometries([cloud],
                                  zoom=0.5,
                                  front=[1, 0, 0],
                                  lookat=center_of_mass,
                                  up=[0, 0, 1])


In [3]:
pcd = o3d.io.read_point_cloud("3dmodels/Black_Sphere.ply")

visualizer(pcd)

In [15]:
threshold = 0.4 # 0 -> 1

x, y, z = np.asarray(pcd.points)[:, :3].T

std = np.std((x, y, z), axis=1) * threshold

hist_x, bins_x = np.histogram(x, bins=50)
hist_y, bins_y = np.histogram(y, bins=50)

max_freq_bin_x = bins_x[np.argmax(hist_x)]
max_freq_bin_y = bins_y[np.argmax(hist_y)]

min_bound = (max_freq_bin_x, max_freq_bin_y, 0.6) - std 
max_bound = (max_freq_bin_x, max_freq_bin_y, 0.7) + (std / 1.9)

bounding_box = o3d.geometry.AxisAlignedBoundingBox(min_bound=min_bound,
                                                    max_bound=max_bound)
axis_bounded_cloud = pcd.crop(bounding_box)

visualizer(axis_bounded_cloud)

In [232]:
def get_min_iter():
    s = 6  #no. of samples used every time to create the shape
    p = 0.99 #99%
    e = 0.5 # 50%
    k = np.log(1-p)/(np.log(1 - np.power((1-e), s)))
    print(k)
    return k
    # k => number of iterations

# get_min_iter()

In [ ]:
# import matplotlib.pyplot as plt
# import numpy as np

# # Generate random data for the histogram
# fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(18, 4))

# axes[0].hist(x, bins=50, color='Yellow', edgecolor='black')
# axes[0].set_title('Histogram 1')
 
# axes[1].hist(y, bins=50, color='Pink', edgecolor='black')
# axes[1].set_title('Histogram 2')

# axes[2].hist(z, bins=50, color='Green', edgecolor='black')
# axes[2].set_title('Histogram 3')

# # Adding labels and title
# for ax in axes:
#     ax.set_xlabel('Values')
#     ax.set_ylabel('Frequency')
#     ax.xaxis.set_major_locator(plt.MaxNLocator(20))  # Adjust the number of ticks here
#     plt.setp(ax.get_xticklabels(), rotation=45)

# plt.tight_layout()
# plt.show()


In [188]:
# import matplotlib.pyplot as plt
# import numpy as np

# plt.hist2d(x, y, bins=30)

# plt.xlabel('X values')
# plt.ylabel('Y values')
# plt.title('2D Histogram (Hexbin Plot)')
# plt.colorbar()
# plt.show()


In [16]:
point = pyrsc.Point()  #Using RANSAC
# s = 1

center, best_inliers_point = point.fit(np.asarray(axis_bounded_cloud.points), thresh=0.7, maxIteration=10)

inlier_cloud_point = axis_bounded_cloud.select_by_index(best_inliers_point)
outlier_cloud_point = axis_bounded_cloud.select_by_index(best_inliers_point, invert=True)

# outlier_cloud_point.paint_uniform_color([1, 0, 0])
# inlier_cloud_point.paint_uniform_color([0, 1, 0])

o3d.visualization.draw_geometries([inlier_cloud_point],
                                    zoom=0.5,
                                    front=[1, 0, 0],
                                    lookat=[0, 0, 0],
                                    up=[0, 0, 1])

[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The requested transformation operation is not supported. 


In [17]:
sphere = pyrsc.Sphere()
# s = 4
center, radius, best_inliers_sphere = sphere.fit(np.asarray(inlier_cloud_point.points), thresh=0.2, maxIteration=75)

inlier_cloud_sphere = inlier_cloud_point.select_by_index(best_inliers_sphere)
outlier_cloud_sphere = inlier_cloud_point.select_by_index(best_inliers_sphere, invert=True)

# outlier_cloud_sphere.paint_uniform_color([1, 0, 0])
# inlier_cloud_sphere.paint_uniform_color([0, 1, 0])

o3d.visualization.draw_geometries([outlier_cloud_sphere, inlier_cloud_sphere],
                                    zoom=0.5,
                                    front=[1, 0, 0],
                                    lookat=[0, 0, 0],
                                    up=[0, 0, 1])

[Open3D WARNING] The number of points is 0 when creating axis-aligned bounding box.
[Open3D WARNING] GLFW Error: WGL: Failed to make context current: The handle is invalid. 


In [22]:
# plane??

sphere = pyrsc.Plane()

# s = 3
equation, best_inliers_plane = sphere.fit(np.asarray(inlier_cloud_sphere.points), thresh=0.07, maxIteration=40)

inlier_cloud_plane = inlier_cloud_sphere.select_by_index(best_inliers_plane)
outlier_cloud_plane = inlier_cloud_sphere.select_by_index(best_inliers_plane, invert=True)

# outlier_cloud_plane.paint_uniform_color([1, 0, 0])
# inlier_cloud_plane.paint_uniform_color([0, 1, 0])

o3d.visualization.draw_geometries([outlier_cloud_plane],
                                    zoom=0.5,
                                    front=[1, 0, 0],
                                    lookat=[0, 0, 0],
                                    up=[0, 0, 1])

In [47]:
best_inliers_plane

array([    0,     1,     2, ..., 36539, 36540, 36541], dtype=int64)

In [23]:
visualizer(inlier_cloud_plane)

In [24]:
pcd = inlier_cloud_plane

In [90]:
def convert_rgb_2_hsv(all_rgb_colors):
    all_hsv_colors = []

    for i in range(len(all_rgb_colors)):
        temp_color = all_rgb_colors[i]
        temp = colorsys.rgb_to_hsv(temp_color[0], temp_color[1], temp_color[2])
        all_hsv_colors.append(temp)

    all_hsv_colors = np.asarray(all_hsv_colors)

    return all_hsv_colors

all_rgb_colors = np.asarray(axis_bounded_cloud.colors)
all_hsv_colors = convert_rgb_2_hsv(all_rgb_colors)


In [96]:
def hsv_filter(hsv_color): #fruit
    # hsv_color is by 0 - 1 range

    # H -> 0 - 179
    low_h = 40/360
    high_h = 100/360
    # strawberry -> H S V
    # min [  0 146 190]
    # max [179 158 214]

    low_h_1 = 325/360
    high_h_1 = 360/360

    # 330 to 30 degree

    # S -> 0,255
    low_s = 0.2 #0.3
    high_s = 1 #/100

    # V -> 0,255
    low_v = 0.2
    high_v = 1

    if ((low_h <= hsv_color[0] <= high_h)) and low_s <= hsv_color[1] <= high_s and low_v <= hsv_color[2] <= high_v:
        return True
    else:
        return False
    

In [98]:
new_points = []
new_colors = []
new_colors_RGB_u = []
fruits = []

new_color_pt = []
new_pt_indexes = []
new_pt_points = []

for index, new_color in enumerate(all_hsv_colors):
    new_point = axis_bounded_cloud.points[index]
    new_color_RGB = all_rgb_colors[index]
    # fruit = fruits[index]
    if not hsv_filter(new_color):
        new_colors_RGB_u.append(new_color_RGB)
        new_points.append(new_point)
        new_colors.append(new_color)
    else:
        new_color_pt.append(new_color)
        new_pt_indexes.append(index)
        new_pt_points.append(new_point)

new_colors_RGB_u = np.asarray(new_colors_RGB_u)
new_colors = np.asarray(new_colors)
new_points = np.asarray(new_points)

filtered_point_cloud = o3d.geometry.PointCloud()
filtered_point_cloud.points = o3d.utility.Vector3dVector(new_points)
filtered_point_cloud.colors = o3d.utility.Vector3dVector(new_colors_RGB_u)


# new_pcd = pcd.select_by_index(new_points, invert=True)
# # new_pcd.points = o3d.utility.Vector3dVector(new_points)
# filtered_point_cloud.colors = o3d.utility.Vector3dVector(new_color_pt)
visualizer(filtered_point_cloud)

In [67]:
# new_pcd = pcd.select_by_index(np.asarray(new_pt_indexes), invert=False)
new_pcd_pointcloud = o3d.geometry.PointCloud()
new_pcd_pointcloud.points = o3d.utility.Vector3dVector(new_pt_points)
# new_pcd.points = o3d.utility.Vector3dVector(new_points)
# new_pcd.colors = o3d.utility.Vector3dVector(new_color_pt)
visualizer(new_pcd_pointcloud)

In [41]:
visualizer(pcd)

In [99]:
cl, ind = filtered_point_cloud.remove_statistical_outlier(nb_neighbors=50,
                                                    std_ratio=0.6)
cl.estimate_normals()
visualizer(cl)


In [75]:
sphere = pyrsc.Sphere()
# s = 3
# removes the soil
_, _,  best_inliers_plane = sphere.fit(np.asarray(cl.points), thresh=0.005, maxIteration=40)

inlier_cloud_plane = cl.select_by_index(best_inliers_plane)
outlier_cloud_plane = cl.select_by_index(best_inliers_plane, invert=True)

# outlier_cloud_plane.paint_uniform_color([1, 0, 0])
# inlier_cloud_plane.paint_uniform_color([0, 1, 0])

# o3d.visualization.draw_geometries([outlier_cloud_plane, inlier_cloud_plane],
#                                     zoom=0.5,
#                                     front=[1, 0, 0],
#                                     lookat=[0, 0, 0],
#                                     up=[0, 0, 1])

visualizer(outlier_cloud_plane)

In [75]:
visualizer(cld)

In [198]:
# cl = outlier_cloud_plane

In [199]:
# sphere = pyrsc.Plane()
# # s = 3
# equation, best_inliers_plane = sphere.fit(np.asarray(cl.points), thresh=0.04, maxIteration=40)

# inlier_cloud_plane = cl.select_by_index(best_inliers_plane)
# outlier_cloud_plane = cl.select_by_index(best_inliers_plane, invert=True)

# # outlier_cloud_plane.paint_uniform_color([1, 0, 0])
# # inlier_cloud_plane.paint_uniform_color([0, 1, 0])

# o3d.visualization.draw_geometries([outlier_cloud_plane],
#                                     zoom=0.5,
#                                     front=[1, 0, 0],
#                                     lookat=[0, 0, 0],
#                                     up=[0, 0, 1])

In [200]:
# filtered_point_cloud, _ = cl.remove_statistical_outlier(nb_neighbors=5, std_ratio=0.5)

# visualizer(filtered_point_cloud)

In [76]:
filtered_point_cloud_r, _ = cld.remove_radius_outlier(nb_points=10, radius=0.01)

visualizer(filtered_point_cloud_r)


In [202]:
def max_red(points):
    # Stricter filter to get just strawberry
    low_h = 0/360
    high_h = 25/360

    low_h_1 = 325/360
    high_h_1 = 360/360

    low_s = 0.5
    high_s = 100/100

    low_v = 0.5
    high_v = 1

    H_mask = ((low_h <= points[:, 0]) & (points[:, 0] <= high_h)) | ((low_h_1 <= points[:, 0]) & (points[:, 0] <= high_h_1))
    S_mask = (low_s <= points[:, 1]) & (points[:, 1] <= high_s)
    V_mask = (low_v <= points[:, 2]) & (points[:, 2] <= high_v)

    final_mask = H_mask & S_mask & V_mask

    return np.sum(final_mask)        

In [82]:
from sklearn.cluster import KMeans

features = np.array(axis_bounded_cloud.points)

kmeans = KMeans(n_clusters=19, random_state=0).fit(features)
red_cluster_index = np.argmin(np.mean(kmeans.cluster_centers_, axis=1))
color = [
    [255, 0, 0],   # Red
    [0, 255, 0],   # Green
    [0, 0, 255],   # Blue
    [255, 255, 0], # Yellow
    [255, 0, 255], # Magenta
    [0, 255, 255], # Cyan
    [0, 128, 0],   # Green (dark)
    [0, 0, 128],   # Navy
    [128, 128, 0],  # Olive
    [128, 0, 128],  # Purple
    [0, 128, 128],  # Teal
    [128, 128, 128],# Gray
    [255, 165, 0],  # Orange
    [255, 192, 203],# Pink
    [0, 0, 0],      # Black
    [128, 0, 0],    # Maroon (dark)
    [0, 128, 0],    # Green (medium)
    [0, 0, 128],
    [128, 128, 128]
]

outlier_cloud_sph = []
final_segments  = []

avg = []

for i in range(len(kmeans.cluster_centers_)):
    temp_p = np.where(kmeans.labels_ == i)
    temp_sphere = axis_bounded_cloud.select_by_index(temp_p[0])
    temp_points = np.asarray(axis_bounded_cloud.colors)[kmeans.labels_ == i]
    # temp_avg = max_red(temp_points)
    # temp_avg = np.average(temp_points, axis=0)
    # avg.append(temp_avg)
    deep_copy_seg = copy.deepcopy(temp_sphere)
    final_segments.append(deep_copy_seg)
    outlier_cloud_sph.append(temp_sphere)
    # temp_sphere.paint_uniform_color(color[i])
    print(i)
    visualizer(temp_sphere)

geometries_to_visualize = [outlier_cloud_sph[i] for i in range(len(kmeans.cluster_centers_))]  # Adjust the range as needed

o3d.visualization.draw_geometries(geometries_to_visualize,
                                    zoom=0.5,
                                    front=[1, 0, 0],
                                    lookat=[0, 0, 0],
                                    up=[0, 0, 1])



0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18


In [1]:
visualizer(geometries_to_visualize[1])
cld2 = geometries_to_visualize[1]

NameError: name 'visualizer' is not defined

In [76]:
def fit_sphere(cld):

    sphere = pyrsc.Sphere()
    # s = 3
    # removes the soil
    _, _,  best_inliers_plane = sphere.fit(np.asarray(cld.points), thresh=0.090, maxIteration=40)

    inlier_cloud_plane = cld.select_by_index(best_inliers_plane)
    outlier_cloud_plane = cld.select_by_index(best_inliers_plane, invert=True)

    outlier_cloud_plane.paint_uniform_color([1, 0, 0])
    inlier_cloud_plane.paint_uniform_color([0, 1, 0])

    # o3d.visualization.draw_geometries([outlier_cloud_plane, inlier_cloud_plane],
    #                                     zoom=0.5,
    #                                     front=[1, 0, 0],
    #                                     lookat=np.mean(np.asarray(cld.points), axis=0),
    #                                     up=[0, 0, 1])

    return len(inlier_cloud_plane.points), len(outlier_cloud_plane.points)

In [100]:
color = [
    [255, 0, 0],   # Red
    [0, 255, 0],   # Green
    [0, 0, 255],   # Blue
    [255, 255, 0], # Yellow
    [255, 0, 255], # Magenta
    [0, 255, 255], # Cyan
    [0, 128, 0],   # Green (dark)
    [0, 0, 128],   # Navy
    [128, 128, 0],  # Olive
    [128, 0, 128],  # Purple
    [0, 128, 128],  # Teal
    [128, 128, 128],# Gray
    [255, 165, 0],  # Orange
    [255, 192, 203],# Pink
    [0, 0, 0],      # Black
    [128, 0, 0],    # Maroon (dark)
    [0, 128, 0],    # Green (medium)
    [0, 128, 128],
    [128, 128, 128]
]

with o3d.utility.VerbosityContextManager(
        o3d.utility.VerbosityLevel.Debug) as cm:
    labels = np.array(
        cl.cluster_dbscan(eps=0.02, min_points=30, print_progress=True))

max_label = labels.max()
outlier_cloud_sph_2 =[]
l = 0
seg_avg = []
for i in range(-1, max_label+1):
    indexes = np.argwhere(labels == i).flatten()
    new_pc = o3d.geometry.PointCloud()
    new_pc.points = o3d.utility.Vector3dVector(np.asarray(cl.points)[indexes])
    new_pc.colors = o3d.utility.Vector3dVector(np.asarray(cl.colors)[indexes])
    if l >= 19:
        l = 0
    # new_pc.paint_uniform_color(color[l])
    l+= 1
    # new_pc.colors = o3d.utility.Vector3dVector(np.asarray(filtered_point_cloud_r.colors)[indexes])
    # temp_avg = np.std(np.asarray(filtered_point_cloud_r.colors)[indexes], axis=0)
    # seg_avg.append(temp_avg)
    outlier_cloud_sph_2.append(new_pc)
    visualizer(new_pc)
    # print(l)
    
o3d.visualization.draw_geometries(outlier_cloud_sph_2,
                                    zoom=0.5,
                                    front=[1, 0, 0],
                                    lookat=[0, 0, 0],
                                    up=[0, 0, 1])


[Open3D DEBUG] Precompute neighbors.
[Open3D DEBUG] Done Precompute neighbors.
[Open3D DEBUG] Compute Clusters
[Open3D DEBUG] Done Compute Clusters: 4


In [101]:
visualizer(outlier_cloud_sph_2[1])
axis_bounded_cloud = outlier_cloud_sph_2[1]

In [41]:
idx = 0
max_points = 0
for i in range(len(outlier_cloud_sph_2)):
    tmp_cld = outlier_cloud_sph_2[i]
    if len(tmp_cld.points) > max_points:
        max_points = len(tmp_cld.points)
        idx = i

print(idx)

3


In [43]:
visualizer(outlier_cloud_sph_2[11])

In [44]:
cld = outlier_cloud_sph_2[11]

In [208]:
def convert_rgb_2_hsv(all_rgb_colors):
    all_hsv_colors_segement = []

    for i in range(len(all_rgb_colors)):
        temp_color = all_rgb_colors[i]
        temp = colorsys.rgb_to_hls(temp_color[0], temp_color[1], temp_color[2]) #bt 0 to 1
        all_hsv_colors_segement.append([temp[0] * 360, temp[1]*100, temp[2]*100])

    all_hsv_colors_segement = np.asarray(all_hsv_colors_segement)

    return all_hsv_colors_segement

all_hsv_colors_segement = convert_rgb_2_hsv(seg_avg)

print(all_hsv_colors_segement)

index = np.argmin(all_hsv_colors_segement, axis=0)[0]

print(index)
# print(outlier_cloud_sph[index])
# print(avg)
# print(np.argmax(avg))
visualizer(outlier_cloud_sph_2[4])


[[ 13.76169311  18.12487989  13.03729584]
 [264.77196007   6.54677815   5.11613097]
 [ 13.91720509  10.0951218    6.64879374]
 [  4.1662796   16.66908272   9.97065576]
 [282.92999538  10.37784015   1.98517612]
 [333.18165748   8.66089264   2.87725673]
 [358.01153293  19.77781879  17.62514963]
 [194.60044911   3.87567736  18.24334462]]
3


In [89]:
import numpy as np

def euclidean_distance(color1, color2):
    return np.linalg.norm(color1 - color2)

# HSV representation of red
red_hsv = np.array([0, 1, 1])

# Calculate the Euclidean distance between each color and red
distances = [euclidean_distance(color, red_hsv) for color in all_hsv_colors_segement]

# Find the index of the color with the smallest distance to red
closest_index = np.argmin(distances)

# Get the closest color
closest_color = all_hsv_colors_segement[closest_index]

print("Closest color to red:", closest_index)


Closest color to red: 9


In [219]:
cld = outlier_cloud_sph_2[idx]

In [220]:
# Get the center of the strawberry
center_strawberry = cld.get_center()

center_pc = o3d.geometry.PointCloud()
center_pc.points = o3d.utility.Vector3dVector(np.asarray([center_strawberry]))
center_pc.paint_uniform_color([0, 0, 1])

o3d.visualization.draw_geometries([cld, center_pc])

In [102]:
# Last Outlier removal if any remain
filtered_point_cloud2, _ = axis_bounded_cloud.remove_statistical_outlier(nb_neighbors=10, std_ratio=0.9)
visualizer(filtered_point_cloud2)

In [105]:
radii = [0.005, 0.01]
cl.estimate_normals()
mesh = o3d.geometry.TriangleMesh.create_from_point_cloud_alpha_shape(
    axis_bounded_cloud, alpha=0.01)

visualizer(mesh)

[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh
[Open3D WARNING] [CreateFromPointCloudAlphaShape] invalid tetra in TetraMesh


In [104]:
# def color_bounding_box(box):
#     # visualizer(mesh)
#     vertices = np.asarray(box.get_box_points())

#     # Define the edges of the oriented bounding box
#     lines = [
#         [0, 1], [1, 2], [2, 3], [3, 0],
#         [4, 5], [5, 6], [6, 7], [7, 4],
#         [0, 4], [1, 5], [2, 6], [3, 7]
#     ]

#     # Create a LineSet to represent the box
#     line_set = o3d.geometry.LineSet(
#         points=o3d.utility.Vector3dVector(vertices),
#         lines=o3d.utility.Vector2iVector(lines)
#     )

#     # Set the color of the box (e.g., red)
#     line_set.colors = o3d.utility.Vector3dVector(np.tile([1, 0, 0], (len(lines), 1)))
#     # o3d.visualization.draw_geometries([mesh, line_set],
#     #                                 zoom=0.5,
#     #                                 front=[1, 0, 0],
#     #                                 lookat=[-0.05272632, -0.02440209, 0.48188474],
#     #                                 up=[0, 0, 1])

#     return line_set

In [105]:
# # AABB box
# box_triangle = mesh.get_axis_aligned_bounding_box()
# line_set_triangle = color_bounding_box(box_triangle)
# o3d.visualization.draw_geometries([mesh, line_set_triangle, center_pc])

In [106]:
# point cloud making convex hull
hull, _ = axis_bounded_cloud.compute_convex_hull()  #cld -> is the point cloud
hull_ls = o3d.geometry.LineSet.create_from_triangle_mesh(hull)
hull_ls.paint_uniform_color((0, 0 , 1))
# visualizer(hull_ls)
o3d.visualization.draw_geometries([hull])

In [109]:
hull.remove_degenerate_triangles()
hull.compute_vertex_normals()
hull.orient_triangles()
o3d.visualization.draw_geometries([axis_bounded_cloud, hull])
volume = hull.get_volume()
print(volume) #0.000596328157324427, 0.0001436078385052907, 0.00014281091173404275

0.0004007525330074498


In [ ]:
# 401 cc -> 4.19 cc (actual) 1cm radius sphere

In [212]:
volume #in m^3??
# CM^3 -> 142.81091173404274741

0.000596328157324427

In [202]:
# whole plant
hull_new, ls_new = pcd.compute_convex_hull()
hull_new.remove_degenerate_triangles()
hull_new.compute_vertex_normals()

o3d.visualization.draw_geometries([pcd, hull_new])
volume = hull_new.get_volume()

In [203]:
volume #whole plant - 4.158153960855363

4.158153960855363

In [214]:
# d = m/v
density = 980 	 #in g/m^3 #1146.43 #in kg/m^3
v_new = 5.963281573244269439e-13
mass = density*v_new
print(mass)

5.844015941779384e-10
